In [58]:
from database.strategy import Strategy
from database.sec import SEC
from database.market import Market
from database.merrill import Merrill
from transformer.date_transformer import DateTransformer
from transformer.column_transformer import ColumnTransformer
from transformer.model_transformer import ModelTransformer
from transformer.product_transformer import ProductTransformer
from transformer.predictor_transformer import PredictorTransformer
from preprocessor.model_preprocessor import ModelPreprocessor
from preprocessor.predictor_preprocessor import PredictorPreprocessor
from modeler.modeler import Modeler as sp
from utils.date_utils import DateUtils
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import math
import requests as r
import os
from dotenv import load_dotenv
load_dotenv()
token=os.getenv("QUANDL")
import pickle

In [59]:
## Loading Constants
start = "2008-01-01"
end = datetime.now().strftime("%Y-%m-%d")
# Loading Databases
strat_db = Strategy("portfolio")
market = Market()
merrill = Merrill()
sec = SEC("sec")
market.connect()
tickers = market.retrieve_data("sp500").sort_values("Symbol")
market.close()
reload = False
model_range = range(len(tickers["Symbol"]))
bonds = ["A","AAA","B","BBB"]
datasets = ["pdr"
#             ,"tiingo"
#             ,"finnhub"
           ]
year = datetime.now().year
quarter = math.ceil((datetime.now().month - 1) // 3) + 1
m = Merrill()
FED = pd.read_csv("FED_metadata.csv")

In [60]:
quarter

2

In [61]:
ds = []
if reload:
    FED["name_simplified"] = [" ".join(row[1]["name"].split(" ")[:5]) for row in FED.iterrows()]
    ml = pd.read_csv("ML_metadata.csv")
    relevant = ml[ml["name"].str.contains("yield")]
    m.connect()
    for code in relevant["code"]:
        name = code.split("E")[0]
        print(code,name)
        url = "https://www.quandl.com/api/v3/datasets/ML/{}".format(code)
        params ={"start_date":"1997-1-1",
                 "end_date":end,
                 "api_key":token}
        data = r.get(url,params)
        yields = pd.DataFrame(data.json()["dataset"]["data"]).rename(columns={0:"date",1:"yield"})
        m.drop_table(name)
        m.store_data(name,yields)
    m.close()   
    merrill.connect()
    for bond in bonds:
        data = merrill.retrieve_data(bond)
        ds.append(data)
    merrill.close()
else:
    merrill.connect()
    for bond in bonds:
        data = merrill.retrieve_data(bond)
        ds.append(data)
    merrill.close()

In [62]:
base = ds[0]
base.rename(columns={"yield":bonds[0]},inplace=True)
for i in range(1,len(bonds)):
    data = ds[i]
    data.rename(columns={"yield":bonds[i]},inplace=True)
    base = base.merge(data.drop("_id",axis=1),on="date",how="left")

In [63]:
base = DateTransformer.convert_to_date("pdr",base,"date")
base["year"] = [x.year for x in base["date"]]
base["quarter"] = [x.quarter for x in base["date"]]
base["week"] = [x.week for x in base["date"]]
bond_weekly = base.groupby(["year","quarter","week"]).mean().reset_index()

In [64]:
sims = []
gap = 7
training_years = 1
sec.connect()
strat_db.connect()
market.connect()
for dataset in datasets:
    strat_db.drop_table("{}_bond_weekly_sim".format(dataset))
    try:
        ## Setting Up
        print(year,quarter)
        date_ranges = DateUtils.create_quarterly_training_range_rec(year,quarter,training_years,gap)
        training_start,training_end,prediction_start,prediction_end = date_ranges
        dates = pd.to_datetime(date_ranges)
        quarters = [x.quarter for x in dates]
        years = [x.year for x in dates]
        models = strat_db.retrieve_data("{}_bond_models".format(dataset))
        print(date_ranges)
        ### switch to all tickers
        for i in tqdm(model_range):
            try:
                ticker = tickers.iloc[i]["Symbol"].replace(".","-")
                price = market.retrieve_price_data("{}_prices".format(dataset),ticker)
                if dataset == "pdr":
                    price = ColumnTransformer.rename_columns(price," ")
                else:
                    price = ColumnTransformer.rename_columns(price,"")
                price = DateTransformer.convert_to_date(dataset,price,"date")
                mt = ModelTransformer(ticker,training_start,training_end,gap)
                pt = ModelTransformer(ticker,prediction_start,prediction_end,gap)
                mr = ModelPreprocessor(ticker)
                prot = ProductTransformer(ticker,prediction_start,prediction_end)
                fundamental_results = models[models["ticker"]==ticker]
                product_refineds = []
                product_qpds = []
                fundamental_results["model"] = [pickle.loads(x) for x in fundamental_results["model"]]
                for i in range(2):
                    fundamental_result = fundamental_results.iloc[i]
                    if fundamental_result["model_type"] == "regression":
                        fundamental_data = bond_weekly.tail(1)
                    else:
                        fundamental_data = bond_weekly.tail(1)
                    product_refined = mr.fundamental_preprocess_rec(fundamental_data.copy())
                    product_refineds.append(product_refined)
                sim = prot.merge_weeklies_v2(product_refineds,fundamental_results.iloc[0:2],year,quarter,"bond")
                strat_db.store_data("{}_bond_weekly_sim".format(dataset),pd.DataFrame(sim).tail(1))
            except Exception as e:
                message = {"status":"quarterly modeling","ticker":ticker,"year":str(year),"quarter":str(quarter),"message":str(e)}
                print(message)
    except Exception as e:
        print(year,quarter,str(e))
sec.close()
market.close()
strat_db.close()

  0%|                                                                                                                                                                                            | 0/506 [00:00<?, ?it/s]

2021 2
['2020-04-01', '2021-03-31', '2021-04-01', '2021-06-30']


 25%|████████████████████████████████████████████▎                                                                                                                                     | 126/506 [00:09<00:24, 15.47it/s]

{'status': 'quarterly modeling', 'ticker': 'CXO', 'year': '2021', 'quarter': '2', 'message': "'t'"}


 86%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                        | 437/506 [00:31<00:04, 14.65it/s]

{'status': 'quarterly modeling', 'ticker': 'TIF', 'year': '2021', 'quarter': '2', 'message': "'t'"}


 97%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████     | 492/506 [00:35<00:00, 15.19it/s]

{'status': 'quarterly modeling', 'ticker': 'WRK', 'year': '2021', 'quarter': '2', 'message': 'single positional indexer is out-of-bounds'}


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 506/506 [00:36<00:00, 13.78it/s]


In [65]:
sim

,year,quarter,week,weekly_bond_classification_prediction,weekly_bond_classification_score,weekly_bond_regression_prediction,weekly_bond_regression_score,ticker
0,2021,2,13,0,0.8,166.615692,0.028311,ZTS
